In [ ]:
from tqdm import tqdm
import os
import time
from random import randint
from keras.callbacks import CSVLogger
import numpy as np
from scipy import stats
import pandas as pd
from keras.utils import np_utils

from keras.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold

import nibabel as nib
import pydicom as pdm
import nilearn as nl
import nilearn.plotting as nlplt
import h5py

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

import seaborn as sns
import imageio
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif
from IPython.display import clear_output
from IPython.display import YouTubeVideo

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import MSELoss


import albumentations as A
from albumentations import Compose, HorizontalFlip
import SimpleITK as sitk
import sys
import os

import warnings
warnings.simplefilter("ignore")


import numpy as np
import pandas as pd
import os
import pydicom
import matplotlib.pyplot as plt
from pydicom.pixel_data_handlers.util import apply_voi_lut
import glob
import cv2
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
from torch import optim
import keras
import warnings
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
np.seterr(divide='ignore', invalid='ignore')
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py
import SimpleITK as sitk
init_notebook_mode(connected=True)
%matplotlib inline

In [ ]:
labels = pd.read_csv("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train_labels.csv")
labels.head()

In [ ]:
data_len = 1000;
labels["imfolder"] = ['{0:05d}'.format(s) for s in labels["BraTS21ID"]]
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/"
labels['path'] = [os.path.join(path ,f) for f in labels["imfolder"]]
train = labels[:data_len]
val_len = int(data_len*0.2)

In [ ]:
train = labels[:0]
path = "../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/"
p = []
d = []
for i in range(1010):
    id_ = '{0:05d}'.format(i)
    if(os.path.exists(path+id_)):
        p.append(path+id_)
        d.append(id_)

In [ ]:
train['path'] = p
train['imfolder'] = d
train.to_csv('test_data.csv')

In [ ]:
def load(path, kind , image_size = 128 , depth =64):
    directory = path +'/'+ kind
#     print(directory)
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    
    image = sitk.GetArrayFromImage(image)

    mid = int(image.shape[0]/2)    
    if image.shape[0]>=64:
        image = image[mid-32:mid+32,:,:]

    image = resize(image, (64, 128, 128), preserve_range=True)    
    image = torch.tensor(image)    
    return image

In [ ]:
image = load(path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/test/01006/',kind = 'FLAIR' )

In [ ]:
image.shape

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(64): 
    plt.subplot(8, 8, i+1)
    plt.imshow(image[i])
    plt.axis("off")

In [ ]:
class BratsDataset(Dataset):
    def __init__(self, df: pd.DataFrame, phase: str="test", is_resize: bool=False):
        self.df = df
        self.phase = phase
        self.augmentations = get_augmentations(phase)
        self.data_types = ['FLAIR', 'T1w', 'T1wCE', 'T2w']
        self.is_resize = is_resize
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        
        id_ = self.df['imfolder'][idx];
        path = self.df['path'][idx];
#         path = 'task2_data_in_nii/00730/'
        id_ = str(id_)
        
        images = []
        for data_type in self.data_types:
            img_path = path
            img = self.load_img(img_path, data_type)#.transpose(2, 0, 1)
            img = img.reshape(64,128,128)   
            img = img.numpy()
#             print(type(img))
            img = self.normalize(img)
            images.append(img)
        img = np.stack(images)
        img = np.moveaxis(img, (0, 1, 2, 3), (0, 3, 2, 1))
        
        if self.phase != "test":
            augmented = self.augmentations(image=img.astype(np.float32), 
                                           )
            
            img = augmented['image']
    
        
            return {
                "Id": id_,
                "image": img,
            }
        
        return {
            "Id": id_,
            "image": img,
        }
    
    def load_img(self, file_path, data_type):
        data = load(file_path , data_type)
#         print(file_path)
#         data = np.asarray(data.dataobj)
        return data
    
    def normalize(self, data: np.ndarray):
        data_min = 0
        
        return (data - data_min) / (np.amax(data) - data_min)
    
    
    
   

In [ ]:
def get_augmentations(phase):
    list_transforms = []
    
    list_trfms = Compose(list_transforms)
    return list_trfms
def get_dataloader(
    dataset: torch.utils.data.Dataset,
    path_to_csv: str,
    phase: str,
    fold: int = 0,
    batch_size: int = 1,
    num_workers: int = 0,
):
    '''Returns: dataloader for the model training'''
    df = pd.read_csv(path_to_csv)
    dataset = dataset(df, phase)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=False,
        shuffle=True,   
    )
    return dataloader

In [ ]:
test_data = pd.read_csv('test_data.csv')

In [ ]:
test_data

In [ ]:
dataloader = get_dataloader(dataset=BratsDataset, path_to_csv='test_data.csv', phase='train', fold=0)
len(dataloader)

data = next(iter(dataloader))
data['Id'], data['image'].shape

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(64): 
    plt.subplot(8, 8, i+1)
    plt.imshow(data['image'][0][0][i])
    plt.axis("off")

In [ ]:
class DoubleConv(nn.Module):
    """(Conv3D -> BN -> ReLU) * 2"""
    def __init__(self, in_channels, out_channels, num_groups=8):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, out_channels, kernel_size=3, stride=1, padding=1),
            #nn.BatchNorm3d(out_channels),
            nn.GroupNorm(num_groups=num_groups, num_channels=out_channels),
            nn.ReLU(inplace=True),

            nn.Conv3d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
            #nn.BatchNorm3d(out_channels),
            nn.GroupNorm(num_groups=num_groups, num_channels=out_channels),
            
            nn.ReLU(inplace=True),
            
          )

    def forward(self,x):
        return self.double_conv(x)

    
class Down(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.MaxPool3d(2, 2),
            DoubleConv(in_channels, out_channels)
        )
    def forward(self, x):
        return self.encoder(x)

    
class Up(nn.Module):

    def __init__(self, in_channels, out_channels, trilinear=False):
        super().__init__()
        
        if trilinear:
            self.up = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose3d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
            
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        diffZ = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        diffX = x2.size()[4] - x1.size()[4]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2, diffZ // 2, diffZ - diffZ // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

    
class Out(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size = 1)

    def forward(self, x):
        return self.conv(x)


class UNet3d(nn.Module):
    def __init__(self, in_channels, n_classes, n_channels):
        super().__init__()
        self.in_channels = in_channels
        self.n_classes = n_classes
        self.n_channels = n_channels

        self.conv = DoubleConv(in_channels, n_channels)
        self.enc1 = Down(n_channels, 2 * n_channels)
        self.enc2 = Down(2 * n_channels, 4 * n_channels)
        self.enc3 = Down(4 * n_channels, 8 * n_channels)
        self.enc4 = Down(8 * n_channels, 8 * n_channels)

        self.dec1 = Up(16 * n_channels, 4 * n_channels)
        self.dec2 = Up(8 * n_channels, 2 * n_channels)
        self.dec3 = Up(4 * n_channels, n_channels)
        self.dec4 = Up(2 * n_channels, n_channels)
        self.out = Out(n_channels, n_classes)

    def forward(self, x):
        x1 = self.conv(x)
        x2 = self.enc1(x1)
        x3 = self.enc2(x2)
        x4 = self.enc3(x3)
        x5 = self.enc4(x4)

        mask = self.dec1(x5, x4)
        mask = self.dec2(mask, x3)
        mask = self.dec3(mask, x2)
        mask = self.dec4(mask, x1)
        mask = self.out(mask)
        return mask

In [ ]:
model = UNet3d(in_channels=4, n_classes=3, n_channels=24)

In [ ]:
model.load_state_dict(torch.load('../input/seg-model/best_model_0.17.pth'))

In [ ]:
model.eval();

In [ ]:
class ShowResult:
  
    def mask_preprocessing(self, mask):
        """
        Test.
        """
        mask = mask.squeeze().cpu().detach().numpy()
        mask = np.moveaxis(mask, (0, 1, 2, 3), (0, 3, 2, 1))

        mask_WT = np.rot90(montage(mask[0]))
        mask_TC = np.rot90(montage(mask[1]))
        mask_ET = np.rot90(montage(mask[2]))

        return mask_WT, mask_TC, mask_ET

    def image_preprocessing(self, image):
        """
        Returns image flair as mask for overlaping gt and predictions.
        """
        image = image.squeeze().cpu().detach().numpy()
        image = np.moveaxis(image, (0, 1, 2, 3), (0, 3, 2, 1))
        flair_img = np.rot90(montage(image[0]))
        return flair_img
    
    def plot(self, image,   prediction):
        image = self.image_preprocessing(image)
#         gt_mask_WT, gt_mask_TC, gt_mask_ET = self.mask_preprocessing(ground_truth)
        pr_mask_WT, pr_mask_TC, pr_mask_ET = self.mask_preprocessing(prediction)
        
        fig, axes = plt.subplots(1, 2, figsize = (35, 30))
    
        [ax.axis("off") for ax in axes]
#         axes[0].set_title("Ground Truth", fontsize=35, weight='bold')
#         axes[0].imshow(image, cmap ='bone')
#         axes[0].imshow(np.ma.masked_where(gt_mask_WT == False, gt_mask_WT),
#                   cmap='cool_r', alpha=0.6)
#         axes[0].imshow(np.ma.masked_where(gt_mask_TC == False, gt_mask_TC),
#                   cmap='autumn_r', alpha=0.6)
#         axes[0].imshow(np.ma.masked_where(gt_mask_ET == False, gt_mask_ET),
#                   cmap='autumn', alpha=0.6)

        axes[1].set_title("Prediction", fontsize=35, weight='bold')
        axes[1].imshow(image, cmap ='bone')
        axes[1].imshow(np.ma.masked_where(pr_mask_WT == False, pr_mask_WT),
                  cmap='cool_r', alpha=0.6)
        axes[1].imshow(np.ma.masked_where(pr_mask_TC == False, pr_mask_TC),
                  cmap='autumn_r', alpha=0.6)
        axes[1].imshow(np.ma.masked_where(pr_mask_ET == False, pr_mask_ET),
                  cmap='autumn', alpha=0.6)

        plt.tight_layout()
        
        plt.show()
        

In [ ]:
train_data = []
tumor = []
ID=[]
pos=[]
for itr, data in enumerate(dataloader):
    print(itr)
    treshold = 0.3
    id_ = int(data['Id'][0]);
    id_ = '{0:05d}'.format(id_)
    print(id_)
    img = data['image']
    logits = model(img)
    test = img.numpy();
    test = test.reshape(4, 128, 128, 64);
    test = np.moveaxis(test, (0, 1, 2, 3), (0, 3, 2, 1))
    tumor.append(test);
    probs = torch.sigmoid(logits)
    show = (probs >= treshold).float()
    predictions = show.numpy()
    predictions = predictions.reshape(3,128,128,64)
    predictions = np.moveaxis(predictions, (0, 1, 2, 3), (0, 3, 2, 1))
#     show_result = ShowResult()
#     show_result.plot(img, show)
    train_data.append(predictions)
    ID.append(id_)
    pos.append(itr)
    if itr==2:
        break;

In [ ]:
long = len(dataloader)

In [ ]:
train_data[0].shape

In [ ]:
plt.figure(figsize=(12, 12))
for i in range(64): 
    plt.subplot(8, 8, i+1)
    plt.imshow(train_data[0][0][i])
    plt.axis("off")

In [ ]:
big = np.array([0]*len(train_data))
for e in range(len(train_data)):
    print(e)
    data = train_data[e]
    area = [0]*64
    print(data[0].shape)
    for i in range (64):
        area[i] = data[0][i].sum()
    max = 0;
    index = 0
    for i in range(64):
        if(area[i]>max):
            max = area[i]
            index = i
    big[e] = index
#     print(area)
    


In [ ]:
big

In [ ]:
ID

In [ ]:
def gaussian_noise(img, mean=0, sigma=0.1):

    # int -> float (標準化)
#     img = img / 255
    # 隨機生成高斯 noise (float + float)
    noise = np.random.rand(img.shape[0], img.shape[1])
#     noise = ;
    # noise + 原圖
#     print(noise)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            if(img[i][j]==0 and noise[i][j]>0.9):
                img[i][j] = img[i][j]+0.8
                
                
    gaussian_out = img
    # 所有值必須介於 0~1 之間，超過1 = 1，小於0 = 0
#     gaussian_out = np.clip(gaussian_out, 0, 1)

    # 原圖: float -> int (0~1 -> 0~255)
#     gaussian_out = np.uint8(gaussian_out*255)
    # noise: float -> int (0~1 -> 0~255)
    noise = np.uint8(noise*255)
    
    return gaussian_out

In [ ]:
plt.imshow(tumor[0][0][42])

In [ ]:
xtrain = np.array([])
for i in range(len(train_data)):
# for i in range(0,1):
    data = train_data[i]
    tumor_data = tumor[i]
#     plt.imshow(tumor_data[0][43])
    id_ = ID[i]
    tumor_data1 = tumor_data[:,:,20:115,20:115]
#     plt.imshow(tumor_data[0][42])
    data1 = data[:,:,20:115,20:115]
    index = big[i]
    
    print(index)
    
    image1 = data1[0][index-3]
    
    tumor_image1 = tumor_data1[0][index-3];
    plt.imshow(tumor_image1)
#     image1 = gaussian_noise(image1)
    image2 = data1[0][index]
    tumor_image2 = tumor_data1[0][index];
    image2 = gaussian_noise(image2)
    for i in range(0, 95):
        for j in range(0,95):
            if image1[i][j] == 1:
                tumor_image1[i][j]+=2;
#             else:
#                 tumor_image1[i][j]-=150
            if(image2[i][j]==1):
                tumor_image2[i][j]+=2;
#             else:
#                 tumor_image2[i][j]-=150
                
                
                
    tumor_image1 = tumor_image1.reshape(1,95,95)
    tumor_image2 = tumor_image2.reshape(1,95,95)
    
    
    data = np.concatenate((tumor_image1, tumor_image2))
    print(data.shape)
    data = data.reshape(1, data.shape[0], data.shape[1], data.shape[2])
    if xtrain.size:
        xtrain = np.concatenate((xtrain, data))
       
    else:
        print("first X")
        xtrain = data
    

In [ ]:
xtrain.shape

In [ ]:
p = xtrain[1]
print(p.shape)
plt.imshow(p[0])


In [ ]:
xtrain = xtrain.reshape(xtrain.shape[0], 95, 95, xtrain.shape[1])

In [ ]:
len(xtrain)

In [ ]:
model = keras.models.load_model("../input/model4/great_model_0.7068965435028076")

In [ ]:
pred = model.predict(xtrain)
pred = np.argmax(pred, axis=1)

In [ ]:
pred

In [ ]:
sub = pd.DataFrame(columns=['BraTS21ID', 'MGMT_value'])
sub['BraTS21ID'] = ID
sub['MGMT_value'] = pred

In [ ]:
sub.to_csv('submission.csv', index = False)

In [ ]:
sub